### Data Ingestion Module

In [1]:
import os
%pwd

'/Users/priyaroy/Documents/Projects/nlp-projects/CommentClassifier/research'

In [2]:
os.chdir("../")
%pwd

'/Users/priyaroy/Documents/Projects/nlp-projects/CommentClassifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.CommentClassifier.constants import *
from src.CommentClassifier.utils.common import read_yaml, create_directories

### Configuration Updates

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from src.CommentClassifier.logging import logger

### Components

In [7]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def downlaod_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"File is downloaded")
        else:
            logger.info(f"File already exits")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    


In [18]:
import yaml
with open("config/config.yaml", "r") as f:
    data = yaml.safe_load(f)
    print(type(data), data)

<class 'dict'> {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/priya-roy/unhealthy-comments-Dataset/raw/refs/heads/main/commentClassification.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/commentClassification', 'tokenizer': 'nltk'}, 'model_trainer': {'model_type': 'logistic_regression', 'solver': 'liblinear', 'penalty': 'l2', 'random_state': 42, 'max_iter': 200, 'class_weight': 'balanced'}, 'model_evaluation': {'metrics': ['accuracy', 'precision', 'recall', 'f1_score'], 'validation_split': 0.2, 'cross_validation': True, 'cv_folds': 5}, 'output': {'model_save_path': 'models/trained_model.pkl', 'vectorizer_save_path': 'models/tfidf_vectorizer.pkl', 'evaluation_report_path': 'reports/evaluation_report.txt', 'log_file_path': 'logs/pipeline.log'}}


In [17]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)

data_ingestion.downlaod_file()
data_ingestion.extract_zip_file()

DEBUG: YAML Content Type: <class 'dict'>
[2025-02-20 07:25:11,949: INFO: common: yaml file: config/config.yaml loaded successfully]
DEBUG: YAML Content Type: <class 'dict'>
[2025-02-20 07:25:11,952: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-20 07:25:11,953: INFO: common: created directory at: artifacts]
[2025-02-20 07:25:11,954: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-20 07:25:11,955: INFO: 227420655: File already exits]
